<a href="https://colab.research.google.com/github/pallaviferrao/HAAnalysis/blob/main/HAAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("/content/input/heart.csv")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
# Scaling
from sklearn.preprocessing import RobustScaler

# Train Test Split
from sklearn.model_selection import train_test_split

# Models
import torch
import torch.nn as nn
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Metrics
from sklearn.metrics import accuracy_score, classification_report, roc_curve

# Cross Validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

print('Packages imported...')

Packages imported...


In [5]:
# creating a copy of df
df1 = df

# define the columns to be encoded and scaled
cat_cols = ['sex','exng','caa','cp','fbs','restecg','slp','thall']
con_cols = ["age","trtbps","chol","thalachh","oldpeak"]

# encoding the categorical columns
df1 = pd.get_dummies(df1, columns = cat_cols, drop_first = True)

# defining the features and target
X = df1.drop(['output'],axis=1)
y = df1[['output']]

# instantiating the scaler
scaler = RobustScaler()

# scaling the continuous featuree
X[con_cols] = scaler.fit_transform(X[con_cols])
print("The first 5 rows of X are")
X.head()


The first 5 rows of X are


,age,trtbps,chol,thalachh,oldpeak,sex_1,exng_1,caa_1,caa_2,caa_3,...,cp_2,cp_3,fbs_1,restecg_1,restecg_2,slp_1,slp_2,thall_1,thall_2,thall_3
0,0.592593,0.75,-0.110236,-0.092308,0.9375,1,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0
1,-1.333333,0.00,0.157480,1.046154,1.6875,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
2,-1.037037,0.00,-0.566929,0.584615,0.3750,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,0.074074,-0.50,-0.062992,0.769231,0.0000,1,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
4,0.148148,-0.50,1.795276,0.307692,-0.1250,0,1,0,0,0,...,0,0,0,1,0,0,1,0,1,0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)
print("The shape of X_train is      ", X_train.shape)
print("The shape of X_test is       ",X_test.shape)
print("The shape of y_train is      ",y_train.shape)
print("The shape of y_test is       ",y_test.shape)

The shape of X_train is       (242, 22)
The shape of X_test is        (61, 22)
The shape of y_train is       (242, 1)
The shape of y_test is        (61, 1)


In [7]:
# instantiating the object and fitting
clf = SVC(kernel='linear', C=1, random_state=42).fit(X_train,y_train)

# predicting the values
y_pred = clf.predict(X_test)

# printing the test accuracy
print("The test accuracy score of SVM is ", accuracy_score(y_test, y_pred))

The test accuracy score of SVM is  0.8688524590163934


In [12]:
X_test

,age,trtbps,chol,thalachh,oldpeak,sex_1,exng_1,caa_1,caa_2,caa_3,...,cp_2,cp_3,fbs_1,restecg_1,restecg_2,slp_1,slp_2,thall_1,thall_2,thall_3
179,0.148148,1.00,0.566929,-1.261538,-0.1250,1,1,1,0,0,...,0,0,0,0,0,1,0,1,0,0
228,0.296296,2.00,0.755906,0.184615,-0.3750,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
111,0.148148,1.00,-1.795276,0.615385,-0.3750,1,0,1,0,0,...,1,0,1,1,0,0,1,0,0,1
246,0.074074,0.20,2.661417,-0.092308,0.6875,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,1
60,1.185185,-1.00,0.393701,-0.707692,-0.5000,0,0,1,0,0,...,1,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,1.037037,0.50,0.220472,-0.215385,0.7500,1,0,0,0,1,...,1,0,0,0,0,1,0,0,0,1
104,-0.370370,-0.05,-0.692913,0.307692,-0.5000,1,0,0,0,0,...,1,0,0,1,0,0,1,0,1,0
300,0.962963,0.70,-0.740157,-0.369231,1.6250,1,0,0,1,0,...,0,0,1,1,0,1,0,0,0,1
193,0.370370,0.75,0.661417,-0.338462,1.2500,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [29]:
def predictVal(testVal):
  yRes = clf.predict(testVal)
  return yRes[0]

dR = [[ 0.14814815,  1.        ,  0.56692913, -1.26153846, -0.125     ,
         1.        ,  1.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ]]
dataF = pd.DataFrame(dR,columns=['age', 'trtbps', 'chol', 'thalachh', 'oldpeak', 'sex_1', 'exng_1',
       'caa_1', 'caa_2', 'caa_3', 'caa_4', 'cp_1', 'cp_2', 'cp_3', 'fbs_1',
       'restecg_1', 'restecg_2', 'slp_1', 'slp_2', 'thall_1', 'thall_2',
       'thall_3'])
predictVal(dataF)



0

In [30]:
from sklearn.externals import joblib
joblib.dump(clf, 'randomfs.pkl')
lr = joblib.load('randomfs.pkl')

ImportError: ignored

In [33]:
import pickle
file = open('important', 'wb')
pickle.dump(clf, file)


In [36]:
# open a file, where you stored the pickled data
file = open('important', 'rb')

# dump information to that file
data = pickle.load(file)
data.predict(dataF)

array([0])

In [27]:
X_test.columns

Index(['age', 'trtbps', 'chol', 'thalachh', 'oldpeak', 'sex_1', 'exng_1',
       'caa_1', 'caa_2', 'caa_3', 'caa_4', 'cp_1', 'cp_2', 'cp_3', 'fbs_1',
       'restecg_1', 'restecg_2', 'slp_1', 'slp_2', 'thall_1', 'thall_2',
       'thall_3'],
      dtype='object')

In [28]:
X_test.values[:1]

array([[ 0.14814815,  1.        ,  0.56692913, -1.26153846, -0.125     ,
         1.        ,  1.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ]])